In [158]:
import torch
from torch import cuda, bfloat16
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import requests

In [159]:
from langchain_community.document_loaders import TextLoader,DirectoryLoader # 文本加载器
from langchain.text_splitter import CharacterTextSplitter # 文本分块器
from langchain_community.embeddings import OllamaEmbeddings # Ollama向量嵌入器
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate # 聊天提示模板
from langchain_community.chat_models import ChatOllama # ChatOllma聊天模型
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser # 输出解析器
import requests

In [160]:
# change url 網址
url = "https://en.itmo.ru/en/viewjep/2/5/Big_Data_and_Machine_Learning.htm"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)
# 加载文件
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [161]:
#webpage loader
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(url)
documents = loader.load()

In [162]:
text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=50)
new_docs = text_splitter.split_documents(documents)

In [163]:
# # in persistent db
# import chromadb
# from chromadb.config import Settings


# client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet",
#                                     persist_directory="db/"
#                                 ))
# collection = client.create_collection(name="Students")

In [164]:

# embeddings = HuggingFaceEmbeddings(
#     model_name='sentence-transformers/all-mpnet-base-v2'
#     #model_name='roberta-large'
# )
# db = Chroma.from_documents(new_docs, embeddings)

In [165]:
# show embeddings vector instances
#embeddings.embed_query('this is an embedding')

In [166]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()
db = Chroma.from_documents(new_docs, embeddings)

In [167]:
# from langchain_community.embeddings import OllamaEmbeddings
# embeddings = OllamaEmbeddings(model="llama3")
# db = Chroma.from_documents(new_docs, embeddings)

In [168]:
# query only from db

query = "what are classes in first semester"
retriever = db.as_retriever(k=5) # can add mmr fetch_k=20, search_type="mmr"

result = retriever.get_relevant_documents(query)


In [170]:
# self-defined reranker
def reranker(query, hits):
    from sentence_transformers import CrossEncoder
    
    # To refine the results, we use a CrossEncoder. A CrossEncoder gets both inputs (input_question, retrieved_question)
    # and outputs a score 0...1 indicating the similarity.
    cross_encoder_model = CrossEncoder("cross-encoder/stsb-roberta-base")

    # Now, do the re-ranking with the cross-encoder
    sentence_pairs = [[query, hit.page_content] for hit in hits]
    similarity_scores = cross_encoder_model.predict(sentence_pairs)
    ranking = np.argsort(similarity_scores)
    
    rerank_result = []
    for i in ranking:
        rerank_result.append(result[i])
    return rerank_result

In [171]:
# rerank

from rerankers import Reranker
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever

ranker = Reranker("mixedbread-ai/mxbai-rerank-base-v1", verbose=0)
compressor = ranker.as_langchain_compressor(k=3)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)


If your model is NOT intended to be ran as a one-label cross-encoder, please reload it and specify the model_type! Otherwise, you may ignore this warning. You may specify `model_type='cross-encoder'` to suppress this warning in the future.
Loading TransformerRanker model mixedbread-ai/mxbai-rerank-base-v1


/Users/mac/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [173]:
retriever = db.as_retriever()
# LLM提示模板
template = """You are an assistant for question-answering tasks. 
   Use the following pieces of retrieved context to answer the question. 
   If you don't know the answer, just say that you don't know. 
   Use three sentences maximum and keep the answer concise.
   Question: {question} 
   Context: {context} 
   Answer:
   """
prompt = ChatPromptTemplate.from_template(template)

In [146]:
# delete 
# db.delete_collection()
# get items in chromadb
# db.get()

In [175]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
        temperature=0,
        max_tokens=800,
        model_kwargs={"top_p": 0, "frequency_penalty": 0, "presence_penalty": 0},
    api_key="sk-yH3c6UN5bNPTEuF7haYlC3a8mpyRuPp28zp5poGyZnUfnZCp",
    base_url="https://api.chatanywhere.tech/v1"
    )
rag_chain = (
        {"context": compression_retriever, "question": RunnablePassthrough()} # 上下文信息
        | prompt
        | llm
        | StrOutputParser()
)
# generation
query = "what are classes in second semester"
print(rag_chain.invoke(query))


In the second semester, classes include Academic English, Applied Artificial Intelligence Part 2, Soft Skills elective discipline, Methodology of Translational Research, Machine Learning, Architecture of Neural Networks for Deep Learning, Technologies and Infrastructure for Big Data, Discrete Modeling/Data Visualization, and Research Internship. These classes cover a range of topics related to data analysis, artificial intelligence, and research methodologies.


In [30]:
rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()} # 上下文信息
        | prompt
        | llm
        | StrOutputParser()
)
# 开始查询&生成
query = "what are classes in second semester"
print(rag_chain.invoke(query))


# using llama as generative model
# llm = ChatOllama(model="llama3", temperature=0)
# rag_chain = (
#         {"context": retriever, "question": RunnablePassthrough()} # 上下文信息
#         | prompt
#         | llm
#         | StrOutputParser()
# )
 
# query = "what are classes in first semester"
# print(rag_chain.invoke(query))


In the second semester, classes include Academic English, Applied Artificial Intelligence, Soft Skills elective discipline, Methodology of Translational Research, Machine Learning, Architecture of Neural Networks for Deep Learning, Technologies and Infrastructure for Big Data, and Research Internship.


In [ ]:
# test for customized embedding model from hugging face

In [36]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM

model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
hf = HuggingFacePipeline(pipeline=pipe)

/Users/mac/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [71]:
# encoder to decoder
# not working
#model_id = 'google/flan-t5-large'# go for a smaller model if you dont have the VRAM
model_id = 't5-base'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)

hf = HuggingFacePipeline(pipeline=pipe)


/Users/mac/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [61]:
# works

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
hf = HuggingFacePipeline(pipeline=pipe)

/Users/mac/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [72]:
from langchain_core.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

False
